In [3]:
from collections import deque
import numpy as np
import pandas as pd
import csv
from pulp import *
from pulp import GLPK
import time
import datetime as dt
from datetime import datetime, timedelta
import time
import math

In [2]:
Data = pd.read_csv('exemple.csv', header=0, parse_dates=[0], index_col=0, squeeze=True)
Data.info
Data.dropna()

,Brigade,Terminal,Compagnie,N parking,N vol,Immat,Heure accostage,heure desaccostage
Date,,,,,,,,
2020-02-23,Journee,T1,AH,W09,6040,JK,8:15,9:10
2020-02-23,Journee,T1,DTH,W12,1903,CB,8:45,10:10
2020-02-23,Journee,T1,DTH,W08,1803,CA,9:55,11:20
2020-02-23,Journee,T1,DTH,W10A,2111,CF,10:05,11:05
2020-02-23,Journee,T1,DTH,W10A,2220,CF,13:50,14:55
2020-02-23,Journee,T1,DTH,W12,2110,CC,14:25,15:15
2020-02-23,Journee,T1,DTH,W10C,1321,CB,14:45,18:10
2020-02-23,Journee,T1,AH,W11A,6006,JU,17:10,18:20
2020-02-23,Journee,T1,AH,W09,6026,KB,17:20,18:25


In [5]:

TimeGap=10
def Trans_to_min(x):
    D=x.day*24*60
    min = x.minute
    H = x.hour *60
    return int(D+min+H)
time= Data["Heure accostage"]
time.dropna()
print(time)
Graph_intervale=[]
for i in [x for x in range(len(time)) if x != None]:
    # print(time[i])
    # if time[i]== None :
    #     time[i].pop()
    # else:
    try:
        Graph_intervale.append([])
        time[i] = dt.datetime.strptime(str(time[i]), '%Y-%d-%m %H:%M:%S')
        min= time[i]- dt.timedelta(minutes=TimeGap//2)
        Graph_intervale[i].append(Trans_to_min(min))
        max= time[i]+ dt.timedelta(minutes=TimeGap//2)
        Graph_intervale[i].append(Trans_to_min(max))
    except:
        time[i] = dt.datetime.strptime(str(time[i]), '%H:%M')
        min= time[i]- dt.timedelta(minutes=TimeGap//2)
        Graph_intervale[i].append(Trans_to_min(min))
        max= time[i]+ dt.timedelta(minutes=TimeGap//2)
        Graph_intervale[i].append(Trans_to_min(max)) 
print((Graph_intervale))


Date
2020-02-23    1900-01-01 08:15:00
2020-02-23    1900-01-01 08:45:00
2020-02-23    1900-01-01 09:55:00
2020-02-23    1900-01-01 10:05:00
2020-02-23    1900-01-01 13:50:00
2020-02-23    1900-01-01 14:25:00
2020-02-23    1900-01-01 14:45:00
2020-02-23    1900-01-01 17:10:00
2020-02-23    1900-01-01 17:20:00
2020-02-23    1900-01-01 17:55:00
2020-02-23    1900-01-01 08:05:00
2020-02-23    1900-01-01 08:16:00
2020-02-23    1900-01-01 08:40:00
2020-02-23    1900-01-01 09:25:00
2020-02-23    1900-01-01 11:10:00
2020-02-23    1900-01-01 12:10:00
2020-02-23    1900-01-01 12:36:00
2020-02-23    1900-01-01 13:55:00
2020-02-23    1900-01-01 17:40:00
2020-02-23    1900-01-01 08:05:00
2020-02-23    1900-01-01 08:20:00
2020-02-23    1900-01-01 08:05:00
2020-02-23    1900-01-01 08:10:00
2020-02-23    1900-01-01 08:20:00
2020-02-23    1900-01-01 08:20:00
2020-02-23    1900-01-01 08:30:00
2020-02-23    1900-01-01 08:55:00
2020-02-23    1900-01-01 09:15:00
2020-02-23    1900-01-01 09:33:00
2020-02-2

C:\Users\pc\AppData\Local\Temp/ipykernel_5492/407279120.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time[i] = dt.datetime.strptime(str(time[i]), '%Y-%d-%m %H:%M:%S')


In [7]:
def Verify_in_interval(M1,M2):
    if M1[0]<=M2[0] and M1[1]>=M2[0]:
        return True
    elif M1[0]<=M2[1] and M1[1]>=M2[1]:
        return True
    else: 
        return False 
def   listAdj(M):
    listAdj=[]
    for i in range(len(M)):
        listAdj.append([])
        for j in [x for x in range(len(M)) if x != i]:
            if Verify_in_interval(M[i],M[j]) or Verify_in_interval(M[j],M[i]) :
                listAdj[i].append(j)
            else:
                pass
    return listAdj
            #print(listAdj[i])
    #print(listAdj[i])
# print(M)
Adj=listAdj(Graph_intervale)
print(Adj)

[[10, 11, 19, 20, 21, 22, 23, 24], [12, 26], [3, 29], [2], [17, 36, 37], [38, 45], [39], [8], [7], [47, 48], [0, 19, 21, 22], [0, 20, 22, 23, 24], [1, 25], [27, 28], [33], [], [], [4, 36, 37], [], [0, 10, 21, 22], [0, 11, 22, 23, 24, 25], [0, 10, 19, 22], [0, 10, 11, 19, 20, 21, 23, 24], [0, 11, 20, 22, 24, 25], [0, 11, 20, 22, 23, 25], [12, 20, 23, 24], [1], [13], [13], [2], [31], [30, 32], [31], [14], [], [], [4, 17, 37], [4, 17, 36], [5], [6, 40], [39], [], [], [], [], [5], [47, 48, 49], [9, 46, 48, 49], [9, 46, 47, 49], [46, 47, 48], []]


In [8]:
def matriceAdj(Adj):
    Mat=[]
    for i in range( len(Adj)):
        Mat.append([])
        for j in range( len(Adj)):
            Mat[i].append(0)
            for k in range(len(Adj[i])):
                if j==Adj[i][k]:
                    Mat[i][j]=1
    return Mat

In [9]:
def arret(M):
    ex=[[],[]]
    for i in range (len(M)-1):
        for j in range(i+1,len(M)):
            if M[i][j]==1:
                ex[0].append(i)
                ex[1].append(j)
    return ex


In [12]:
Mat=matriceAdj(Adj)
Mat= np.array(Mat)
print (Mat)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]


In [13]:
Ex=arret(Mat)
print("ex1= ",Ex[0])
print("ex2= ",Ex[1])

ex1=  [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 4, 4, 4, 5, 5, 6, 7, 9, 9, 10, 10, 10, 11, 11, 11, 11, 12, 13, 13, 14, 17, 17, 19, 19, 20, 20, 20, 20, 21, 22, 22, 23, 23, 24, 30, 31, 36, 39, 46, 46, 46, 47, 47, 48]
ex2=  [10, 11, 19, 20, 21, 22, 23, 24, 12, 26, 3, 29, 17, 36, 37, 38, 45, 39, 8, 47, 48, 19, 21, 22, 20, 22, 23, 24, 25, 27, 28, 33, 36, 37, 21, 22, 22, 23, 24, 25, 22, 23, 24, 24, 25, 25, 31, 32, 37, 40, 47, 48, 49, 48, 49, 49]


In [19]:
def Gloton(Adj):
    n=len(Adj)
    NC=dict()
    C=[0 for k in range(n)]
    for i in range(n):
        NC[i]=[]
    def  rajouter(k,NC):
        if ((k in NC)==True):
            return NC
        else:
            NC.append(k)
            return NC
    i=0
    while i<=n-1:
        exist=True
        k=0
        while (exist==True):
            exist=k in NC[i]
            if (exist==True):
                k+=1
        C[i]=k    
        for j in range(len(Adj[i])):
            voisin=Adj[i][j]
            NC[voisin]=rajouter(k,NC[voisin])
        i+=1
    return C
C=Gloton(Adj)
print(C)

[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 2, 2, 3, 4, 3, 5, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 2, 3, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 2, 3, 0]


In [20]:
# %%
# Import of the pyomo module
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo.environ

# Creation of a Concrete Model
model = ConcreteModel()

# %%
model.C = Set(initialize=range(6), doc='les Conducteur')
model.J = Set(initialize=range(len(Adj)), doc='les jobs')
model.A = Set(initialize=range(len(Ex[1])), doc='les arcs')

# %%
model.n = Param(initialize=8, doc='le nombre de jobs')
model.c = Param(initialize=6, doc='le nombre de conducteur')
model.m = Param(initialize=25, doc='les relations entre les arcs')
model.ex1 = Param(model.A,initialize=Ex[0], doc='extrémité 1')
model.ex2 = Param(model.A,initialize=Ex[1], doc='extrémité 2')

# %%
model.x = Var(model.C, model.J,domain=Binary, doc='affectation conducteur')
model.v = Var(model.C,bounds=(0,None),domain=Integers, doc='les taches')
model.y = Var(bounds=(0,None),domain=Integers, doc='valeur de comparaison')

# %%
def Cotraint1(model,j):
  return sum(model.x[i,j] for i in model.C) == 1
model.C1 = Constraint(model.J, rule=Cotraint1, doc='verification daffectation')


def Contraint2(model,i,k):
  return ((model.x[i,model.ex1[k]]+model.x[i,model.ex2[k]])) <= 1  

model.C2 = Constraint (model.C,model.J, rule=Contraint2, doc='verification de non chevauchement ')

def Contraint3(model,i):
  return sum(model.x[i,j] for j in model.J)==model.v[i]

model.C3 = Constraint (model.C, rule=Contraint3, doc='une seul tache par conducteur')

def Contraint4(model,i):
  return (model.v[i]<=model.y)

model.C4 = Constraint (model.C, rule=Contraint4, doc='maximiser')

# %%
def objective_rule(model):
  return model.y
model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define objective function')

# %%
def pyomo_postprocess(options=None, instance=None, results=None):
  model.x.display()
# %%


opt = SolverFactory("gurobi")
results = opt.solve(model,tee=True)
#sends results to stdout
results.write()
print("\nDisplaying Solution\n" + '-'*60)
pyomo_postprocess(None, model, results)
# %%




Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-10
Read LP format model from file C:\Users\pc\AppData\Local\Temp\tmp2m7b5hx5.pyomo.lp
Reading time = 0.01 seconds
x314: 370 rows, 314 columns, 1243 nonzeros
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 370 rows, 314 columns and 1243 nonzeros
Model fingerprint: 0xd6620f0a
Variable types: 1 continuous, 313 integer (306 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 12.0000000
Presolve removed 181 rows and 7 columns
Presolve time: 0.02s
Presolved: 189 rows, 307 columns, 948 nonzeros
Variable types: 0 continuous, 307 integer (306 binary)

Root relaxation: objective 8.500000e+00, 105 iterations, 0.00 seconds (0.00 work units)

    Nodes    | 

In [22]:
x=model.x
Group_co=[]
for i in model.C:
    Group_co.append([])
    for j in model.J:
        if x[i,j].value == 1:
            Group_co[i].append(j)

print(Group_co)
for i in range(len(Group_co)):
    print("list des vols du conducteur "+str(i+1))
    for j in Group_co[i]:
        print(time[j])


[[0, 2, 9, 25, 28, 33, 42, 44, 49], [8, 18, 22, 26, 27, 29, 31, 41], [3, 7, 12, 13, 14, 16, 20, 35, 36], [17, 19, 24, 32, 38, 39, 45, 47, 50], [1, 4, 10, 11, 15, 34, 43], [5, 6, 21, 23, 30, 37, 40, 46, 48]]
list des vols du conducteur 1
1900-01-01 08:15:00
1900-01-01 09:55:00
1900-01-01 17:55:00
1900-01-01 08:30:00
1900-01-01 09:33:00
1900-01-01 11:20:00
1900-01-01 15:48:00
1900-01-01 16:30:00
1900-01-01 18:06:00
list des vols du conducteur 2
1900-01-01 17:20:00
1900-01-01 17:40:00
1900-01-01 08:10:00
1900-01-01 08:55:00
1900-01-01 09:15:00
1900-01-01 09:53:00
1900-01-01 10:30:00
1900-01-01 15:35:00
list des vols du conducteur 3
1900-01-01 10:05:00
1900-01-01 17:10:00
1900-01-01 08:40:00
1900-01-01 09:25:00
1900-01-01 11:10:00
1900-01-01 12:36:00
1900-01-01 08:20:00
1900-01-01 13:13:00
1900-01-01 13:57:00
list des vols du conducteur 4
1900-01-01 13:55:00
1900-01-01 08:05:00
1900-01-01 08:20:00
1900-01-01 10:40:00
1900-01-01 14:15:00
1900-01-01 14:55:00
1900-01-01 14:30:00
1900-01-01 17